##### Run this cell to set your notebook up (only mandatory if rlss2019-docker image is not used)

In [ ]:
!git clone https://github.com/yfletberliac/rlss2019-hands-on.git > /dev/null 2>&1

# Reinforcement Learning - Practical Session 1


## Review

A Markov Decision Process (MDP) is defined as tuple $(S, A, P, r, \gamma)$ where:
* $S$ is the state space
* $A$ is the action space 
* $P$ represents the transition probabilities, $P(s,a,s')$ is the probability of arriving at state $s'$ by taking action $a$ in state $s$
* $r$ is the reward function such that $r(s,a,s')$ is the reward obtained by taking action $a$ in state $s$ and arriving at $s'$
* $\gamma$ is the discount factor

A deterministic policy $\pi$ is a mapping from $S$ to $A$: $\pi(s)$ is the action to be taken at state $s$.

The goal of an agent is to find the policy $\pi$ that maximizes the expected sum of discounted rewards by following $\pi$. The value of $\pi$ is defined as

$$
V_\pi(s) = E\left[ \sum_{t=0}^\infty \gamma^t r(S_t, A_t, S_{t+1}) | S_0 = s \right]
$$

$V_\pi(s)$ and the optimal value function, defined as $V^*(s) = \max_\pi V_\pi(s)$, can be shown to satisfy the Bellman equations:

$$
V_\pi(s) = \sum_{s' \in S}  P(s,\pi(s),s')[r(s,\pi(s),s') + \gamma V_\pi(s')]
$$


$$
V^*(s) = \max_{a\in A} \sum_{s' \in S}  P(s,a,s')[r(s,a,s') + \gamma V^*(s')]
$$

It is sometimes better to work with Q functions:

$$
Q_\pi(s, a) = \sum_{s' \in S}  P(s,a,s')[r(s,a,s') + \gamma  Q^*(s', \pi(s')]
$$


$$
Q^*(s, a) = \sum_{s' \in S}  P(s,a,s')[r(s,a,s') + \gamma \max_{a'} Q^*(s', a')]
$$

such that $V_\pi(s) = Q_\pi(s, \pi(s))$ and $V^*(s) = \max_a Q^*(s, a)$.


### Using value iteration to compute an optimal policy
If the reward function and the transition probabilities are known (and the state and action spaces are not very large), we can use dynamic programming methods to compute $V^*(s)$. Value iteration is one way to do that.


#####  Value iteration to compute $V^*(s)$
$$
T^* Q(s,a) = \sum_{s'}P(s'|s,a)[ r(s, a, s') + \gamma \max_{a'} Q(s', a')]   \\
$$


* For any $Q_0$, let $Q_n = T^* Q_{n-1}$. 
* We have $\lim_{n\to\infty}Q_n = Q^*$ and $Q^* = T^* Q^*$


##### Finding the optimal policy from $V^\pi(s)$

The optimal policy $\pi^*$ can be computed as

$$
\pi^*(s) \in \arg\max_{a\in A} Q^*(s, a) =  \arg\max_{a\in A} \sum_{s' \in S}  P(s,a,s')[r(s,a,s') + \gamma V^*(s')]
$$

###  Q-Learning and SARSA 

When the reward function and the transition probabilities are *unknown*, we cannot use dynamic programming to find the optimal value function. Q-Learning and SARSA are stochastic approximation algorithms that allows us to estimate the value function by using only samples from the environment.

#####  Q-learning

The Q-Learning algorithm allows us to estimate the optimal Q function using only trajectories from the MDP obtained by following some exploration policy. 

Q-learning with $\varepsilon$-greedy exploration does the following update at time $t$:

1. In state $s_t$, take action $a_t$  such that $a_t$ is random with probability $\varepsilon$ and $a_t \in \arg\max_a \hat{Q}_t(s_t,a) $ with probability $1-\varepsilon$;
2. Observe $s_{t+1}$ and reward $r_t$;
3. Compute $\delta_t = r_t + \gamma \max_a \hat{Q}_t(s_{t+1}, a) - \hat{Q}_t(s_t, a_t)$;
4. Update $\hat{Q}_{t+1}(s, a) = \hat{Q}_t(s, a) + \alpha_t(s,a)\delta_t\mathbb{1}\{s=s_t, a=a_t\}  $


##### SARSA

SARSA is similar to Q-learning, but it is an *on-policy* algorithm: it follows a (stochastic) policy $\pi_Q$ and updates its estimate towards the value of this policy. One possible choice is:

$$
\pi_Q(a|s) = \frac{ \exp(\tau^{-1}Q(s,a))  }{\sum_{a'}\exp(\tau^{-1}Q(s,a')) }
$$
where $\tau$ is a "temperature" parameter: when $\tau$ approaches 0, $\pi_Q(a|s)$ approaches the greedy (deterministic) policy $a \in \arg\max_{a'}Q(s,a')$.

At each time $t$, SARSA keeps an estimate $\hat{Q}_t$ of the true Q function and uses $\pi_{\hat{Q}_t}(a|s)$ to choose the action $a_t$. If $\tau \to 0$ with a proper rate as $t \to \infty$, $\hat{Q}_t$ converges to $Q$ and $\pi_{\hat{Q}_t}(a|s)$ converges to the optimal policy $\pi^*$. 

The SARSA update at time $t$ is done as follows:

1. In state $s_t$, take action $a_t \sim \pi_{\hat{Q}_t}(a|s_t)$ ;
2. Observe $s_{t+1}$ and reward $r_t$;
3. Sample the next action $a_{t+1} \sim \pi_{\hat{Q}_t}(a|s_{t+1})$;
4. Compute $\delta_t = r_t + \gamma \hat{Q}_t(s_{t+1}, a_{t+1}) - \hat{Q}_t(s_t, a_t)$
5. Update $\hat{Q}_{t+1}(s, a) = \hat{Q}_t(s, a) + \alpha_t(s,a)\delta_t\mathbb{1}\{s=s_t, a=a_t\}$

## Goals

Your goal is to implement Value Iteration, Q-Learning and SARSA for the [Frozen Lake](https://gym.openai.com/envs/FrozenLake-v0/) environment.

* In exercise 1, you will implement the Bellman operators $T^\pi$ and $T^*$ and verify their properties.
* In exercise 2, you will implement value iteration
* In exercises 3 and 4, you will implement Q-Learning and SARSA


In [2]:
import sys
sys.path.insert(0, './rlss2019-hands-on/utils')
# If using the Docker image, replace by:
# sys.path.insert(0, '../utils')

import numpy as np
from scipy.special import softmax # for SARSA
import matplotlib.pyplot as plt
from frozen_lake import FrozenLake
from test_env import ToyEnv1

# FrozenLake environment

(You can use ToyEnv1 to debug your algorithms)

In [3]:
# Creating an instance of FrozenLake
# --- If deterministic=False, transitions are stochastic. Try both cases!
#env = FrozenLake(gamma=0.95, deterministic=False, data_path="../data") 

# Small environment for debugging
env = ToyEnv1(gamma=0.95)

# Useful attributes
print("Set of states:", env.states)
print("Set of actions:", env.actions)
print("Number of states: ", env.Ns)
print("Number of actions: ", env.Na)
print("P has shape: ", env.P.shape)  # P[s, a, s'] = env.P[s, a, s']
print("discount factor: ", env.gamma)
print("")

# Usefult methods
state = env.reset() # get initial state
print("initial state: ", state)
print("reward at (s=1, a=3,s'=2): ", env.reward_func(1,3,2))
print("")

# A random policy
policy = np.random.randint(env.Na, size = (env.Ns,))
print("random policy = ", policy)

# Interacting with the environment
print("(s, a, s', r):")
for time in range(4):
    action = policy[state]
    next_state, reward, done, info = env.step(action)
    print(state, action, next_state, reward)
    if done:
        break
    state = next_state
print("")

# Visualizing the environment
try:
    env.render()
except:
    pass # render not available

Set of states: [0, 1, 2]
Set of actions: [0, 1]
Number of states:  3
Number of actions:  2
P has shape:  (3, 2, 3)
discount factor:  0.95

initial state:  0
reward at (s=1, a=3,s'=2):  1.0

random policy =  [0 1 0]
(s, a, s', r):
0 0 1 0.0
1 1 2 1.0
2 0 1 0.0
1 1 0 0.0



# Exercise 1: Bellman operator

1. Write a function that takes an environment and a state-action value function $Q$ as input and returns the Bellman optimality operator applied to $Q$, $T^* Q$ and the greedy policy with respect to $Q$.
3. Let $Q_1$ and $Q_2$ be state-action value functions. Verify the contraction property:  $\Vert T^* Q_1 - T^* Q_2\Vert \leq \gamma ||Q_1 - Q_2||$, where $||V|| = \max_{s,a} |Q(s,a)|$.

In [3]:
# --------------
# Your answer to 1.
# --------------
def bellman_operator(Q, env):
    TQ = 0
    greedy_policy = []
    ###
    # To fill
    ###
    return TQ, greedy_policy

In [4]:
# --------------
# Your answer to 2.
# --------------
print("Contraction of Bellman operator")

Contraction of Bellman operator


# Exercise 2: Value iteration

1. (Optimal Value function) Write a function that takes as input an initial state-action value function `Q0` and an environment `env` and returns a vector `Q` such that $||T^* Q -  Q ||_\infty \leq \varepsilon $ and the greedy policy with respect to $Q$.
2. Test the convergence of the function you implemented.

In [5]:
# --------------
# Your answer to 1.
# --------------
def value_iteration(Q0, env, epsilon=1e-5):
    """
    Finding the optimal value function. To be done!
    """
    TQ = 0
    greedy_policy = []
    return TQ, greedy_policy

In [6]:
# --------------
# Your answer to 2.
# --------------

# Exercise 3: Q-Learning

#####  Q-learning

The Q-Learning algorithm allows us to estimate the optimal Q function using only trajectories from the MDP obtained by following some exploration policy. 

Q-learning with $\varepsilon$-greedy exploration does the following update at time $t$:

1. In state $s_t$, take action $a_t$  such that $a_t$ is random with probability $\varepsilon$ and $a_t \in \arg\max_a \hat{Q}_t(s_t,a) $ with probability $1-\varepsilon$ (**act function**);
2. Observe $s_{t+1}$ and reward $r_t$ (**step in the environment**);
3. Compute $\delta_t = r_t + \gamma \max_a \hat{Q}_t(s_{t+1}, a) - \hat{Q}_t(s_t, a_t)$ (**to be done in .optimize()**) ;
4. Update $\hat{Q}_{t+1}(s, a) = \hat{Q}_t(s, a) + \alpha_t(s,a)\delta_t\mathbb{1}\{s=s_t, a=a_t\}$ (**in optimize too**)


Implement Q-learning and test its convergence.

In [7]:
#-------------------------------
# Q-Learning implementation
# ------------------------------

class QLearning:
    """
    Implements Q-learning algorithm with epsilon-greedy exploration
    """
    def __init__(self, env, gamma, learning, epsilon): # You can add more argument to your init (lr decay, eps decay)
        pass
    
    def act(state, greedy=False, ...): # You don't have to use this template for your algorithm, those are just hints
        """
        Takes a state as input and outputs an action (acting greedily or not with respect to the q function)
        """
        pass
    
    def optimize(state, action_taken, next_state, reward, ...):
        """
        Takes (s, a, s', r) as input and optimize the Q function
        """
        pass

In [8]:
# ---------------------------
# Convergence of Q-Learning
# ---------------------------

# Number of Q learning iterations
n_steps = int(1e5)  
#n_steps = 10

Q0 = np.zeros((env.Ns, env.Na))
# You can use Q_opt from value iteration to check the correctness of q learning
Q_opt, pi_opt = value_iteration(Q0, env, epsilon=1e-6)
#       ^ and the optimal policy too

# Exercise 4: SARSA

SARSA is similar to Q-learning, but it is an *on-policy* algorithm: it follows a (stochastic) policy $\pi_Q$ and updates its estimate towards the value of this policy. One possible choice is:

$$
\pi_Q(a|s) = \frac{ \exp(\tau^{-1}Q(s,a))  }{\sum_{a'}\exp(\tau^{-1}Q(s,a')) }
$$
where $\tau$ is a "temperature" parameter: when $\tau$ approaches 0, $\pi_Q(a|s)$ approaches the greedy (deterministic) policy $a \in \arg\max_{a'}Q(s,a')$.

At each time $t$, SARSA keeps an estimate $\hat{Q}_t$ of the true Q function and uses $\pi_{\hat{Q}_t}(a|s)$ to choose the action $a_t$. If $\tau \to 0$ with a proper rate as $t \to \infty$, $\hat{Q}_t$ converges to $Q$ and $\pi_{\hat{Q}_t}(a|s)$ converges to the optimal policy $\pi^*$. 

The SARSA update at time $t$ is done as follows:

1. In state $s_t$, take action $a_t \sim \pi_{\hat{Q}_t}(a|s_t)$ ;
2. Observe $s_{t+1}$ and reward $r_t$;
3. Sample the next action $a_{t+1} \sim \pi_{\hat{Q}_t}(a|s_{t+1})$;
4. Compute $\delta_t = r_t + \gamma \hat{Q}_t(s_{t+1}, a_{t+1}) - \hat{Q}_t(s_t, a_t)$
5. Update $\hat{Q}_{t+1}(s, a) = \hat{Q}_t(s, a) + \alpha_t(s,a)\delta_t\mathbb{1}\{s=s_t, a=a_t\}$

In [9]:
#-------------------------------
# SARSA implementation
# ------------------------------

class Sarsa:
    """
    Implements SARSA algorithm.
    """
    def __init__(self, env, gamma, learning_rate=None, tau=1.0): # Again, those are suggestions, you can add more arguments
        pass
    def act():
        pass
    def optimize():
        pass


In [10]:
# ---------------------------
# Convergence of SARSA
# ---------------------------

# Create SARSA object
sarsa = Sarsa(env, gamma=env.gamma)

# Again, you can use Q_opt and pi_opt from value_iteration to check sarsa's convergence. 

How those two algorithms behave ? 
Do both of them find the optimal policy ?